## Problem Description
A company's marketing team needs to determine the products to market to each customer in a way that maximizes the marketing campaign return on investment while considering the following constraints:

limits on funding available for the campaign.
restrictions on the minimum number of product offers that can be made in a campaign.
campaign return-on-investment hurdle rates that must be met.

### We are giving sample Data for the model

### Dataset generation

Generate the data with 100, 1000 and 10000 customers with expected profits and costs sampled from the list above.

Generate a Gaussian Distribution for expected profits and costs for the customers


## Naive Formulation with fixed Budget $B$

$i \in I$: Customer Index.

$j \in J$: Index of products.

### Parameters
$r_{i,j}$: Expected profit given that customer $i \in I$ is offered product $j \in J$.

$\nu_{i,j}$: Average variable cost associated with the offer of product  $j \in J$ to customer $i \in I$.

$Q_{j}$: Minimum number of offers of product $j \in J$. 

$R$: Corporate hurdle rate. Hurdle rate is defined as the minimum return on investment for the marketing campaign to be vaiable.

$B$: Total marketing campaign budget.

### Decision Variables
$y_{k,j} \geq 0$: Customer $i \in I$ is offered product $j \in J$.


### Objective Function
- **Total profit**. Maximize total expected profit from marketing campaign and heavily penalize any correction to the budget.

\begin{equation}
\text{Max} \quad Z = \sum_{i \in I} \sum_{j \in J} r_{i,j} \cdot y_{i,j}
\tag{0}
\end{equation}

### Constraints


- **Budget**. The total marketing budget is limited by a fixed value $B$ .

\begin{equation}
\sum_{i \in I} \sum_{j \in J} \nu_{i,j} \cdot y_{i,j} \leq B
\tag{2}
\end{equation}

- **Number of offers**. Maximum number of products for each customer should be less than or equal to one.

\begin{equation}
\sum_{j \in J} y_{i,j} \leq 1 \quad \forall i \in i
\tag{1}
\end{equation}


- **Offers limit**. Minimum number of offers of each product.

\begin{equation}
\sum_{i \in I} y_{i,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}

- **ROI**. The minimum ROI constraint ensures that the ratio of total profits over cost is at least one plus the corporate hurdle rate.

\begin{equation}
\sum_{i \in I} \sum_{j \in J} r_{i,j} \cdot y_{i,j} \geq (1+R) \cdot \sum_{i \in I} \sum_{j \in J} \nu_{i,j} \cdot y_{i,j}
\tag{4}
\end{equation}

where, 
$x_{i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J$  is offered to customer $i \in I$, and 0 otherwise.

### Scalars

The corporate hurdle-rate (return on investment) is twenty percent ($R = 0.20$).

The budget available for the marketing campaign is $\$3000$.

### Decision Variables
$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

### Constraints

- **Budget**

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B
\tag{2}
\end{equation}

Where

$y_{i,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$\nu_{i,j}$: Variable cost associated with the offer of product  $j \in J$ to customer $i \in I$.

$B$: Marketing campaign budget.

- **Offers limit**. Minimum number of offers of each product.

\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}


- **ROI**.

\begin{equation}
\sum_{i \in I} \sum_{j \in J} r_{i,j} \cdot y_{i,j} \geq (1+R) \cdot \sum_{i \in I} \sum_{j \in J} \nu_{i,j} \cdot y_{i,j}
\tag{4}
\end{equation}

Where

$y_{i,j} \geq 0$: Customer $i \in I$ that are offered product $j \in J$.

$r_{i,j}$: Expected profit from the offer of product $j \in J$ to customer $i \in I$.

$\nu_{i,j}$: Variable cost associated with the offer of product  $j \in J$ to customer $i \in I$.

$R$: Corporate hurdle rate.

**Number of offers**. Maximum number of offers of products for each person should be less than or equal to one.

\begin{equation}
\sum_{j \in J} y_{i,j} \leq 1 \quad \forall i \in i
\tag{1}
\end{equation}

### Objective Function
- **Total profit**

\begin{equation}
\text{Max} \quad Z = \sum_{i \in I} \sum_{j \in J} r_{i,j} \cdot y_{i,j}
\tag{0}
\end{equation}

Where

$y_{i,j} \geq 0$: Customer $i \in I$ offered product $j \in J$.

$r_{i,j}$: Expected profit from product $j \in J$ to customer $i \in I$.

In [ ]:
# packages installation

%pip install gurobipy
%pip install scikit-learn

In [ ]:
# package imports

import gurobipy as gp
from gurobipy import GRB
import numpy as np
from sklearn.cluster import KMeans

from matplotlib import pyplot as plt
import random
import scipy.stats
import pandas as pd

In [ ]:
# set randomization seed

random.seed(147)
np.random.seed(147)

In [ ]:
#Change exp costs and exp profits to numpy arrays

def get_variables(n ,m):
    
    expected_profits_lb = 1000
    expected_profits_ub = 4000

    costs_per_customer_lb = 80
    costs_per_customer_ub = 450
    
    cust_costs = np.zeros(shape=(m, n))
    cust_exp_profits = np.zeros(shape=(m, n))
    cust_prods = np.zeros(shape=(m, n), dtype=object)
    products = np.zeros(shape=(n), dtype=object)
    
    for p_i in range(n):
        products[p_i] = "p"+str(p_i)
        for c_j in range(m):
            cust_prods[c_j][p_i] = ('c'+ str(c_j), products[p_i])
            cust_costs[c_j][p_i] = random.randrange(costs_per_customer_lb, costs_per_customer_ub, 40)
            cust_exp_profits[c_j][p_i] = random.randrange(expected_profits_lb, expected_profits_ub, 200)
            
        
    return (products, cust_prods, cust_exp_profits, cust_costs)

In [ ]:
def build_naive_model(model, corporate_rate, cust_prods_keys, expected_cost, budget, expected_profit, customers, products, min_offers):
    ### Decisions variables (cust_prod_keys is either 1 or 0)
    y = model.addVars(cust_prods_keys, vtype=GRB.BINARY, name="assign")
    
    # Constraints
    const_budget = model.addConstr((y.prod(expected_cost) <= budget), name='budget')
    const_minOffers = model.addConstrs( (y.sum('*',j) >= min_offers[j] for j in products), name='min_offers')
    ROI_con = model.addConstr((y.prod(expected_profit) - (1 + corporate_rate)*y.prod(expected_cost) >= 0), name='ROI')
    no_offers = model.addConstrs( (y.sum('c'+str(i),'*') == 1 for i in range(customers)), name='no_offers')
    
    # Objective Function
    model.setObjective(y.prod(expected_profit), GRB.MAXIMIZE)
    model.write('naive_cust_assign.lp')
    model.optimize()

In [ ]:
# Product Count
n = [5, 10, 15]

# Customer Count 
m = [100, 200, 300, 1000, 2000, 10000]

# Corporate Hurdle Rate
R = [0.05, 0.1, 0.15]

In [ ]:
# Declare and initialize model
mt = gp.Model('Naive')

run_times = {}
for num_products, customers, corporate_rate in np.array(np.meshgrid(n ,m , R)).T.reshape(-1,3):
    num_products = int(num_products)
    customers = int(customers)
    print(f'\n\n{num_products}, {customers}, {corporate_rate}\n\n')
    ##Assuming that atleast 75% of the products should be sold for the campaign to be feasible
    q_sum = customers * 0.75

    products, cust_prods, cust_exp_profits, cust_costs = get_variables(num_products, customers)

    products_min_quant = np.random.multinomial(q_sum, np.ones(len(products))/len(products), size=1)[0]

    cust_costs_f = cust_costs.flatten()
    cust_prods_f = cust_prods.flatten()
    cust_exp_profits_f = cust_exp_profits.flatten()

    budget = (np.sum(cust_costs_f)/cust_costs_f.size)*customers

    print("Budget: {}".format(budget))

    #Expected profits from the customers
    cust_prods_keys, expected_profit = gp.multidict(dict(zip(cust_prods_f, cust_exp_profits_f)))

    #Costs for the customers
    cust_prods_keys, expected_cost = gp.multidict(dict(zip(cust_prods_f, cust_costs_f)))

    #Minimum quantities of products to be sold for the campaign to be viable
    products, min_offers = gp.multidict(dict(zip(products, products_min_quant)))

    build_naive_model(mt, corporate_rate ,cust_prods_keys, expected_cost, budget, expected_profit, customers, products, min_offers)
    
    print(mt.runTime)
    run_times[(num_products, customers, corporate_rate)] = mt.runTime

### Creating Clusters

In the second approach, we group the customers into clusters and relax the binary constriant of $y_{i,j}. This will relax the problem from MLP to LP. 

We are using KMeans to cluster the data and use the expected_profits to cluster the customers. We also expirement with
expected_costs and expected_profits/expected_costs as well.


In [ ]:
cluster_sizes = [5, 10, 15]
num_clusters = cluster_sizes[1]
clusters = ["k"+str(i) for i in range(num_clusters)]
km = KMeans(
       init="random",
       n_clusters=num_clusters,
       n_init=10,
       max_iter=300,
       random_state=42
      )

## Tactical formulation with fixed budget $B$

### Part 1 : Allocate the number of products for each cluster

$k \in K$: Index of clusters.

$j \in J$: Index of products.

### Parameters
$r_{k,j}$: Expected profit given that customer in cluster $k \in K$ is offered product $j \in J$.

$\nu_{k,j}$: Average variable cost associated with the offer of product  $j \in J$ to customer of cluster $k \in K$.
  
$N_{k}$: Number of customers in cluster $k \in K$.

$Q_{j}$: Minimum number of offers of product $j \in J$. 

$R$: Corporate hurdle rate. Hurdle rate is defined as the minimum return on investment for the marketing campaign to be vaiable.

$B$: Total marketing campaign budget.

### Decision Variables
$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.


### Objective Function
- **Total profit**. Maximize total expected profit from marketing campaign and heavily penalize any correction to the budget.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{j \in J} r_{k,j} \cdot y_{k,j}
\tag{0}
\end{equation}

### Constraints

- **Number of offers**. Maximum number of offers of products for each cluster should be less than the number of customers in the cluster.

\begin{equation}
\sum_{j \in J} y_{k,j} \leq N_{k} \quad \forall k \in K
\tag{1}
\end{equation}

- **Budget**. The total marketing budget is limited by a fixed value $B$ .

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B
\tag{2}
\end{equation}

- **Offers limit**. Minimum number of offers of each product.

\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}

- **ROI**. The minimum ROI constraint ensures that the ratio of total profits over cost is at least one plus the corporate hurdle rate.

\begin{equation}
\sum_{k \in K} \sum_{j \in J} r_{k,j} \cdot y_{k,j} \geq (1+R) \cdot \sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j}
\tag{4}
\end{equation}



### Expected profit

### Expected cost

The expected cost of offering a product to a customer in a cluster is calculated below.


### Number of customers

### Minimum number of offers

### Scalars

The corporate hurdle-rate (return on investment) is twenty percent ($R = 0.20$).

The budget available for the marketing campaign is $\$3000$.


### Decision Variables
$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$z \geq 0$: Increase in budget in order to have a feasible campaign.

### Constraints

- **Number of offers**

\begin{equation}
\sum_{j \in J} y_{k,j} \leq N_{k} \quad \forall k \in K
\tag{1}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$N_{k}$: Number of customers in cluster $k \in K$.

- **Budget**

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B
\tag{2}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$\nu_{k,j}$: Average variable cost associated with the offer of product  $j \in J$ to an average customer of cluster $k \in K$.

$B$: Marketing campaign budget.

**Offers limit**
\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$Q_{j}$: Minimum number of offers of product $j \in J$.


- **ROI**.

\begin{equation}
\sum_{k \in K} \sum_{j \in J} r_{k,j} \cdot y_{k,j} \geq (1+R) \cdot \sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j}
\tag{4}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$r_{k,j}$: Expected profit to the bank from the offer of product $j \in J$ to an average customer of cluster $k \in K$.

$\nu_{k,j}$: Average variable cost associated with the offer of product  $j \in J$ to an average customer of cluster $k \in K$.

$R$: Corporate hurdle rate.

### Objective Function
- **Total profit**

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{j \in J} r_{k,j} \cdot y_{k,j}
\tag{0}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$r_{k,j}$: Expected profit from product $j \in J$ to customer of cluster $k \in K$.

### Part 2: Assigning the products to the customers within each cluster while maintaing cluster constraints. 


Once the optimal values $y_{k,j}$, for all $j \in J$ and $k \in K$, of the Part 1 have been found, we should determine which individual customers in cluster $k$ should get an offer of a product. The optimal way to do that is to solve an assignment problem using the estimated expected profit for the individual customers.


### Sets and Indices
$i \in I^{k}$: Index and set of customers in cluster $k \in K$.

$j \in J^{k}$: Index and subset of products offered to customers in cluster $k \in K$ , where $J^{k} = \{ j \in J: y_{k,j} > 0 \}$ .

### Parameters

$r_{k,i,j}$: Expected individual profit of customer $i \in I^{k}$ from  offer of product $j \in J^{k}$. 

$Y_{k,j} = \lfloor y_{k,j} \rfloor $: Number of customers in cluster k that will get an offer of product $j \in J^{k}$.

### Decision Variables
$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.



### Objective Function
- **Total profit**. Maximize total individual profit.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}


### Constraints

- **Product offers**. Allocate offers of a product to customers of each cluster.

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}


- **Offers limit**. At most one product may be offered to a customer of a cluster.

\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

- **Binary constraints**. Either a product offer is given to a customer of cluster k or not.

\begin{equation}
x_{k,i,j} \in \{0,1 \} \quad \forall i \in I^{k},  j \in J^{k}, k \in K
\tag{3}
\end{equation}


### Customer expected profit

### Customer expected costs

### Decision Variables
$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

### Constraints

- **Product offers**. Allocate offers of a product to customers of each cluster.

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}

Where

$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

$Y_{k,j} = \lfloor y_{k,j} \rfloor $: Number of customers in cluster k that will get an offer of product $j \in J^{k}$.




- **Offers limit**. At most one product may be offered to a customer of a cluster.

\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

Where

$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

### Objective Function

- **Total profit**. Maximize total individual expected profit.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}

Where

$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

$r_{k,i,j}$: Expected individual profit of customer $i \in I^{k}$ from  offer of product $j \in J^{k}$. 



In [ ]:
# Declare and initialize model
mt_tact = gp.Model('Tactical')

def build_tactical_model(corporate_rate, cluster_prods_keys, number_customers, clusters, products, expected_cost, expected_profit):

    ### Decisions variables
    y = mt_tact.addVars(cluster_prods_keys, name="allocate")

    # Constraints

    # Constraint on number of offers at each cluster

    const_maxOffers = mt_tact.addConstrs((y.sum(k,'*') <= number_customers[k]  for k in clusters), name='maxOffers')
    # Budget constraint

    const_budget = mt_tact.addConstr((y.prod(expected_cost) <= budget), name='budget')
    const_minOffers = mt_tact.addConstrs( (y.sum('*',j) >= min_offers[j] for j in products), name='min_offers')
    # Constraint to ensure minimum ROI

    ROI_con = mt_tact.addConstr((y.prod(expected_profit) - (1 + corporate_rate)*y.prod(expected_cost) >= 0), name='ROI')
    mt_tact.setObjective(y.prod(expected_profit), GRB.MAXIMIZE)
    mt_tact.write('tact_max_profit.lp')
    mt_tact.optimize()
    
    return y

In [ ]:
m_assign = gp.Model('Assignment')

def build_operational_model(y, ccp, clusters, products, cluster_cust_map, customer_profit):
    # Declare and initialize model

    ### Decision variables

    x = m_assign.addVars(ccp, vtype=GRB.BINARY, name="assign")

    productOffers = {}

    for k in clusters:
        for j in products:
                productOffers[k,j] = m_assign.addConstr(gp.quicksum(x[k,i,j] for kk,i,jj in ccp if (kk ==k and jj == j)) == 
                                                  round(y[k,j].x), name='prodOffers_' + str(k) + ',' + str(j) )

    # check = (gp.quicksum(x[k,i,j] for kk,ii,j in ccp if (kk == k and ii == i) ) <= 1 for k,i in cluster_cust_map)

    check2 = (x.sum(k,i,'*') <= 1 for k,i in cluster_cust_map)
    customerOffers = m_assign.addConstrs(check2, name ='custOffers')

    ### Objective function

    # Maximize total profit

    m_assign.setObjective(x.prod(customer_profit), GRB.MAXIMIZE)

    # Verify model formulation

    m_assign.write('tact_assign_model.lp')

    # Run optimization engine

    m_assign.optimize()
    
    return x

In [ ]:
##Take expected profits as the feature to group the customers
##Todo: Experiment with expected costs and expected_profit/expected_cost

# Keeping corporate Cost as 0.15
corporate_rate = 0.15

tactical_model_results = {}
operational_model_results = {}

for num_products, customers in np.array(np.meshgrid(n ,m)).T.reshape(-1,2):

    (products, cust_prods, cust_exp_profits, cust_costs) = get_variables(num_products, customers)
    cust_clust = km.fit_predict(cust_exp_profits)
    clusters_exp_profit = km.cluster_centers_
    exp_profit_cluster = np.array([None for k in range(num_clusters)])
    exp_cost_cluster = np.array([None for k in range(num_clusters)])

    clust_cust_exp_profit = {} #Maintain the cluster to customer to product profit mapping
    clust_cust_exp_cost = {} #Maintain the cluster to customer to product cost mapping
    cluster_cust_map = [] #Maintain the cluster to customer mapping

    for i in range(len(cust_clust)):

        cluster_cust_map.append(('k'+str(cust_clust[i]), 'c'+str(i)))
        if(exp_profit_cluster[cust_clust[i]]) is None:
            exp_profit_cluster[cust_clust[i]] = np.array(cust_exp_profits[i])
        else:
            exp_profit_cluster[cust_clust[i]] = np.vstack([exp_profit_cluster[cust_clust[i]], cust_exp_profits[i]])

        for j in range(len(products)):
            clust_cust_exp_profit['k'+str(cust_clust[i]),'c'+str(i),products[j]] = cust_exp_profits[i][j]

        if(exp_cost_cluster[cust_clust[i]]) is None:
            exp_cost_cluster[cust_clust[i]] = np.array(cust_costs[i])
        else:
            exp_cost_cluster[cust_clust[i]] = np.vstack([exp_cost_cluster[cust_clust[i]], cust_costs[i]])

        for j in range(len(products)):
            clust_cust_exp_cost['k'+str(cust_clust[i]),'c'+str(i),products[j]] = cust_costs[i][j]


    clusters_exp_cost = np.array([np.mean(cluster, axis = 0) for cluster in exp_cost_cluster.flatten()])
    print(clusters_exp_cost.shape)

    #Expected profits from the customers
    cluster_prods = [('k'+str(i), prod) for i in range(num_clusters) for prod in products]
    clusters_exp_profits = [clusters_exp_profit[i][j] for i in range(num_clusters) for j in range(len(products))]

    #Costs for the customers
    clusters_exp_costs = [clusters_exp_cost[i][j] for i in range(num_clusters) for j in range(len(products))]

    clusters = ['k'+str(i) for i in range(num_clusters)]
    number_customers = [len(exp_profit_cluster[i]) for i in range(num_clusters)]

    cluster_prods_keys, expected_profit = gp.multidict(dict(zip(cluster_prods, clusters_exp_profits)))
    cluster_prods_keys, expected_cost = gp.multidict(dict(zip(cluster_prods, clusters_exp_costs)))
    clusters, number_customers = gp.multidict(dict(zip(clusters, number_customers)))
    #Minimum quantities of products to be sold for the campaign to be viable
    products, min_offers = gp.multidict(dict(zip(products, products_min_quant)))

    y = build_tactical_model(corporate_rate, cluster_prods_keys, number_customers, clusters, products, expected_cost, expected_profit)
    tactical_model_results[(num_products, customers)] = [mt_tact.runTime, mt_tact.getObjective()]
    print(y)
    
    ccp, customer_profit = gp.multidict(clust_cust_exp_profit)
    ccp, customer_cost = gp.multidict(clust_cust_exp_cost)
    x = build_operational_model(y, ccp, clusters, products, cluster_cust_map, customer_profit)
    operational_model_results[(num_products, customers)] = [m_assign.runTime, m_assign.getObjective()]